# 📦 Product Demand Forecasting
This notebook demonstrates a complete demand forecasting pipeline using:
- **EDA** – Exploratory Data Analysis
- **Classical Models** – SARIMA, Exponential Smoothing (Holt-Winters)
- **Machine Learning** – XGBoost with time-series features
- **Evaluation** – MAE, RMSE, MAPE
- **Visualization** – Forecast plots with confidence intervals

## 1. Install & Import Libraries

In [ ]:
# Uncomment to install if needed
# !pip install pandas numpy matplotlib seaborn scikit-learn statsmodels xgboost pmdarima

import warnings
warnings.filterwarnings('ignore')

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.dates as mdates
import seaborn as sns

from statsmodels.tsa.statespace.sarimax import SARIMAX
from statsmodels.tsa.holtwinters import ExponentialSmoothing
from statsmodels.tsa.seasonal import seasonal_decompose
from statsmodels.tsa.stattools import adfuller

from sklearn.metrics import mean_absolute_error, mean_squared_error
from sklearn.preprocessing import LabelEncoder
import xgboost as xgb

# Plotting style
plt.rcParams['figure.figsize'] = (14, 5)
plt.rcParams['axes.grid'] = True
plt.rcParams['grid.alpha'] = 0.3
sns.set_palette('tab10')

RANDOM_STATE = 42
np.random.seed(RANDOM_STATE)
print('Libraries loaded successfully ✅')

## 2. Generate / Load Data
We simulate realistic multi-product demand data with **trend**, **seasonality**, **promotions**, and **noise**.
Replace the cell below with `pd.read_csv('your_file.csv')` if you have real data.

In [ ]:
def generate_demand_data(n_products=3, start='2021-01-01', periods=156):
    """Simulate weekly demand for multiple products."""
    dates = pd.date_range(start=start, periods=periods, freq='W')
    products = [f'Product_{chr(65+i)}' for i in range(n_products)]
    
    rows = []
    for prod in products:
        base     = np.random.randint(200, 600)
        trend    = np.linspace(0, np.random.randint(50, 200), periods)
        seasonal = base * 0.25 * np.sin(2 * np.pi * np.arange(periods) / 52)
        promo    = np.random.choice([0, base * 0.3], size=periods, p=[0.88, 0.12])
        noise    = np.random.normal(0, base * 0.08, periods)
        demand   = np.maximum(0, base + trend + seasonal + promo + noise)
        
        for i, d in enumerate(dates):
            rows.append({
                'date'      : d,
                'product'   : prod,
                'demand'    : round(demand[i]),
                'price'     : round(np.random.uniform(10, 50), 2),
                'on_promo'  : int(promo[i] > 0)
            })
    return pd.DataFrame(rows)

df = generate_demand_data()
df['date'] = pd.to_datetime(df['date'])
df = df.sort_values(['product', 'date']).reset_index(drop=True)
print(f'Dataset shape: {df.shape}')
df.head(10)

## 3. Exploratory Data Analysis (EDA)

In [ ]:
print('=== Basic Statistics ===')
print(df.groupby('product')['demand'].describe().round(2))

In [ ]:
# Demand over time per product
fig, axes = plt.subplots(len(df['product'].unique()), 1, figsize=(14, 10), sharex=True)

for ax, (prod, grp) in zip(axes, df.groupby('product')):
    ax.plot(grp['date'], grp['demand'], linewidth=1.5, label=prod)
    promo_dates = grp[grp['on_promo'] == 1]['date']
    ax.scatter(promo_dates, grp[grp['on_promo'] == 1]['demand'],
               color='red', s=30, zorder=5, label='Promo')
    ax.set_ylabel('Units Sold')
    ax.set_title(prod)
    ax.legend(loc='upper left', fontsize=8)

axes[-1].xaxis.set_major_formatter(mdates.DateFormatter('%b %Y'))
plt.suptitle('Weekly Demand by Product', fontsize=14, fontweight='bold', y=1.01)
plt.tight_layout()
plt.show()

In [ ]:
# Correlation heatmap and distribution
fig, axes = plt.subplots(1, 2, figsize=(14, 4))

# Distribution
for prod, grp in df.groupby('product'):
    axes[0].hist(grp['demand'], bins=30, alpha=0.6, label=prod)
axes[0].set_title('Demand Distribution')
axes[0].set_xlabel('Units Sold')
axes[0].legend()

# Correlation
corr = df[['demand', 'price', 'on_promo']].corr()
sns.heatmap(corr, annot=True, fmt='.2f', ax=axes[1], cmap='coolwarm', center=0)
axes[1].set_title('Feature Correlation')

plt.tight_layout()
plt.show()

## 4. Time Series Decomposition

In [ ]:
# Focus on one product for decomposition
FOCUS_PRODUCT = 'Product_A'
series = df[df['product'] == FOCUS_PRODUCT].set_index('date')['demand']

decomposition = seasonal_decompose(series, model='additive', period=52)

fig, axes = plt.subplots(4, 1, figsize=(14, 12))
for ax, component, title in zip(
    axes,
    [series, decomposition.trend, decomposition.seasonal, decomposition.resid],
    ['Observed', 'Trend', 'Seasonal', 'Residual']
):
    ax.plot(component, linewidth=1.2)
    ax.set_title(title, fontweight='bold')

plt.suptitle(f'Seasonal Decomposition — {FOCUS_PRODUCT}', fontsize=14, fontweight='bold', y=1.01)
plt.tight_layout()
plt.show()

In [ ]:
# Augmented Dickey-Fuller stationarity test
adf_result = adfuller(series.dropna())
print(f'ADF Statistic : {adf_result[0]:.4f}')
print(f'p-value       : {adf_result[1]:.4f}')
print(f'Stationary    : {"Yes ✅" if adf_result[1] < 0.05 else "No ❌ (consider differencing)"}')

## 5. Train / Test Split

In [ ]:
TEST_WEEKS = 12   # Forecast horizon

product_data = {}
for prod, grp in df.groupby('product'):
    grp = grp.set_index('date').sort_index()
    train = grp.iloc[:-TEST_WEEKS]
    test  = grp.iloc[-TEST_WEEKS:]
    product_data[prod] = {'train': train, 'test': test}
    print(f'{prod}: train={len(train)} weeks, test={len(test)} weeks')

## 6. Model 1 — Holt-Winters Exponential Smoothing

In [ ]:
def fit_holt_winters(train_series, test_len, seasonal_periods=52):
    model = ExponentialSmoothing(
        train_series,
        trend='add',
        seasonal='add',
        seasonal_periods=seasonal_periods
    ).fit(optimized=True)
    forecast = model.forecast(test_len)
    return forecast

hw_forecasts = {}
for prod, data in product_data.items():
    hw_forecasts[prod] = fit_holt_winters(
        data['train']['demand'], TEST_WEEKS
    )
print('Holt-Winters fitted ✅')

## 7. Model 2 — SARIMA

In [ ]:
def fit_sarima(train_series, test_len, order=(1,1,1), seasonal_order=(1,1,1,52)):
    model = SARIMAX(
        train_series,
        order=order,
        seasonal_order=seasonal_order,
        enforce_stationarity=False,
        enforce_invertibility=False
    ).fit(disp=False)
    forecast = model.forecast(steps=test_len)
    return forecast, model

sarima_forecasts = {}
sarima_models    = {}
for prod, data in product_data.items():
    print(f'Fitting SARIMA for {prod}...')
    fc, mdl = fit_sarima(data['train']['demand'], TEST_WEEKS)
    sarima_forecasts[prod] = fc
    sarima_models[prod]    = mdl
print('SARIMA fitted ✅')

## 8. Model 3 — XGBoost with Engineered Features

In [ ]:
def create_features(df_prod):
    """Create time-series features from date index."""
    df_f = df_prod.copy()
    df_f['week']        = df_f.index.isocalendar().week.astype(int)
    df_f['month']       = df_f.index.month
    df_f['quarter']     = df_f.index.quarter
    df_f['year']        = df_f.index.year
    df_f['week_of_year']= df_f.index.isocalendar().week.astype(int)
    df_f['trend']       = np.arange(len(df_f))
    # Lag features
    for lag in [1, 2, 4, 8, 12, 52]:
        df_f[f'lag_{lag}'] = df_f['demand'].shift(lag)
    # Rolling stats
    for w in [4, 8, 12]:
        df_f[f'roll_mean_{w}'] = df_f['demand'].shift(1).rolling(w).mean()
        df_f[f'roll_std_{w}']  = df_f['demand'].shift(1).rolling(w).std()
    return df_f.dropna()

FEATURE_COLS = [
    'week', 'month', 'quarter', 'year', 'trend', 'price', 'on_promo',
    'lag_1', 'lag_2', 'lag_4', 'lag_8', 'lag_12', 'lag_52',
    'roll_mean_4', 'roll_mean_8', 'roll_mean_12',
    'roll_std_4',  'roll_std_8',  'roll_std_12'
]

xgb_forecasts = {}
xgb_models    = {}

for prod, data in product_data.items():
    full_feat  = create_features(pd.concat([data['train'], data['test']]))
    train_feat = full_feat.iloc[:-TEST_WEEKS]
    test_feat  = full_feat.iloc[-TEST_WEEKS:]

    X_train, y_train = train_feat[FEATURE_COLS], train_feat['demand']
    X_test           = test_feat[FEATURE_COLS]

    model = xgb.XGBRegressor(
        n_estimators=300,
        learning_rate=0.05,
        max_depth=5,
        subsample=0.8,
        colsample_bytree=0.8,
        random_state=RANDOM_STATE,
        verbosity=0
    )
    model.fit(X_train, y_train,
              eval_set=[(X_test, test_feat['demand'])],
              verbose=False)

    xgb_forecasts[prod] = pd.Series(
        model.predict(X_test), index=test_feat.index
    )
    xgb_models[prod] = (model, FEATURE_COLS)

print('XGBoost fitted ✅')

## 9. Ensemble Forecast

In [ ]:
# Simple average ensemble
ensemble_forecasts = {}
for prod in product_data:
    hw  = hw_forecasts[prod].values
    sar = sarima_forecasts[prod].values
    xgb_= xgb_forecasts[prod].values
    idx = product_data[prod]['test'].index
    ensemble_forecasts[prod] = pd.Series(
        (hw + sar + xgb_) / 3, index=idx
    )
print('Ensemble computed ✅')

## 10. Evaluation Metrics

In [ ]:
def mape(actual, predicted):
    actual, predicted = np.array(actual), np.array(predicted)
    mask = actual != 0
    return np.mean(np.abs((actual[mask] - predicted[mask]) / actual[mask])) * 100

def evaluate(actual, predicted, model_name):
    mae  = mean_absolute_error(actual, predicted)
    rmse = np.sqrt(mean_squared_error(actual, predicted))
    map_ = mape(actual, predicted)
    return {'Model': model_name, 'MAE': round(mae, 2),
            'RMSE': round(rmse, 2), 'MAPE (%)': round(map_, 2)}

all_results = []
for prod in product_data:
    actual = product_data[prod]['test']['demand']
    for name, fc in [
        ('Holt-Winters', hw_forecasts[prod].values),
        ('SARIMA',       sarima_forecasts[prod].values),
        ('XGBoost',      xgb_forecasts[prod].values),
        ('Ensemble',     ensemble_forecasts[prod].values),
    ]:
        res = evaluate(actual.values, fc, name)
        res['Product'] = prod
        all_results.append(res)

results_df = pd.DataFrame(all_results)[['Product', 'Model', 'MAE', 'RMSE', 'MAPE (%)']]
print('=== Evaluation Results ===')
print(results_df.to_string(index=False))

In [ ]:
# Bar chart of MAPE by model
pivot = results_df.pivot(index='Model', columns='Product', values='MAPE (%)')
pivot.plot(kind='bar', figsize=(10, 5), edgecolor='black', alpha=0.85)
plt.title('MAPE (%) by Model and Product — Lower is Better', fontweight='bold')
plt.ylabel('MAPE (%)')
plt.xticks(rotation=0)
plt.legend(title='Product')
plt.tight_layout()
plt.show()

## 11. Forecast Visualisation

In [ ]:
for prod, data in product_data.items():
    train = data['train']['demand']
    test  = data['test']['demand']

    fig, ax = plt.subplots(figsize=(14, 5))
    ax.plot(train.index[-52:], train.values[-52:], label='Train (last year)',
            color='steelblue', linewidth=1.8)
    ax.plot(test.index,  test.values,  label='Actual',
            color='black', linewidth=2, linestyle='--')
    ax.plot(test.index,  hw_forecasts[prod].values,
            label='Holt-Winters', color='orange',    linewidth=1.5)
    ax.plot(test.index,  sarima_forecasts[prod].values,
            label='SARIMA',       color='purple',    linewidth=1.5)
    ax.plot(test.index,  xgb_forecasts[prod].values,
            label='XGBoost',      color='green',     linewidth=1.5)
    ax.plot(test.index,  ensemble_forecasts[prod].values,
            label='Ensemble',     color='red',       linewidth=2.5, linestyle='-.')

    ax.axvline(test.index[0], color='grey', linestyle=':', linewidth=1.5, label='Forecast Start')
    ax.set_title(f'Demand Forecast — {prod}', fontweight='bold', fontsize=13)
    ax.set_ylabel('Units Sold')
    ax.legend(loc='upper left', fontsize=8)
    ax.xaxis.set_major_formatter(mdates.DateFormatter('%b %Y'))
    plt.tight_layout()
    plt.show()

## 12. Feature Importance (XGBoost)

In [ ]:
fig, axes = plt.subplots(1, len(xgb_models), figsize=(16, 5))

for ax, (prod, (model, feats)) in zip(axes, xgb_models.items()):
    importance = pd.Series(model.feature_importances_, index=feats)
    importance.sort_values().tail(12).plot(kind='barh', ax=ax, edgecolor='black')
    ax.set_title(f'Feature Importance\n{prod}', fontweight='bold')
    ax.set_xlabel('Importance Score')

plt.tight_layout()
plt.show()

## 13. Future Forecast (Next 8 Weeks)

In [ ]:
FUTURE_WEEKS = 8

for prod, data in product_data.items():
    full_series = df[df['product'] == prod].set_index('date')['demand']

    # Holt-Winters future forecast
    hw_model = ExponentialSmoothing(
        full_series, trend='add', seasonal='add', seasonal_periods=52
    ).fit(optimized=True)
    future_fc = hw_model.forecast(FUTURE_WEEKS)
    future_dates = pd.date_range(
        full_series.index[-1] + pd.Timedelta(weeks=1),
        periods=FUTURE_WEEKS, freq='W'
    )
    future_fc.index = future_dates

    fig, ax = plt.subplots(figsize=(12, 4))
    ax.plot(full_series.index[-52:], full_series.values[-52:],
            label='Historical', color='steelblue', linewidth=1.8)
    ax.plot(future_fc.index, future_fc.values,
            label='8-Week Forecast', color='red', linewidth=2, linestyle='--', marker='o')
    ax.fill_between(future_fc.index,
                    future_fc.values * 0.9,
                    future_fc.values * 1.1,
                    alpha=0.2, color='red', label='±10% CI')
    ax.set_title(f'Future Demand Forecast — {prod} (Next {FUTURE_WEEKS} Weeks)',
                 fontweight='bold')
    ax.set_ylabel('Units Sold')
    ax.legend()
    ax.xaxis.set_major_formatter(mdates.DateFormatter('%b %Y'))
    plt.tight_layout()
    plt.show()

    print(f'\n{prod} — Next {FUTURE_WEEKS} Week Forecast (Holt-Winters):')
    print(future_fc.rename('Forecasted Demand').to_frame().to_string())

## 14. Summary

| Step | What was done |
|------|---------------|
| EDA | Trend, seasonality, promo effects explored |
| Decomposition | Additive decomposition + ADF test |
| Holt-Winters | Triple exponential smoothing |
| SARIMA | Statistical seasonal ARIMA |
| XGBoost | Lag + rolling + calendar features |
| Ensemble | Simple average of all three |
| Evaluation | MAE, RMSE, MAPE per product |
| Future Forecast | 8-week outlook with confidence interval |

**Tips for production use:**
- Replace simulated data with your real sales file
- Use `pmdarima.auto_arima` for automatic SARIMA order selection
- Add external regressors (weather, holidays, stock levels)
- Implement cross-validation with `TimeSeriesSplit`
- Consider Prophet or NeuralProphet for complex seasonality